# Main Notebook Modelo LIGHTGBM Rossman Kaggle
Disclaimer: los resultados que arroja la siguiente notebook corrida en anaconda son inferiores y distintos a los mostrados en kaggle porque esas corridas se realizaron en collab. 

## Import de los modulos necesarios

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import pandas as pd
import numpy as np
import datetime
#from pandas_summary import DataFrameSummary

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-e612b94002b1>", line 1, in <module>
    from tensorflow.compat.v1 import ConfigProto
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\tensorflow\__init__.py", line 101, in <module>
    from tensorflow_core import *
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\tensorflow_core\__init__.py", line 40, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\tensorflow\__init__.py", line 50, in __getattr__
    module = self._load()
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\si

## Load data procesada utilizando las notebooks entregadas

In [ ]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 347
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-9723ca5e5331>", line 1, in <module>
    df = pd.read_feather('train_normalized_data.fth')
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppDa

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-9723ca5e5331>", line 1, in <module>
    df = pd.read_feather('train_normalized_data.fth')
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.p

In [ ]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_bool_fw', 'StateHoliday_bool_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']
contin_vars = ['CompetitionDistance', 
               'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC', 'Precipitationmm',
               'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
               'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
               'AfterStateHoliday_bool', 'BeforeStateHoliday_bool', 'Promo', 'SchoolHoliday', 'StateHoliday_bool']

In [ ]:
# Split data into train/val and define X and y variables
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')

y_out_columns = ['Sales']
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 348
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-6651847425cc>", line 2, in <module>
    df_train = df[df.Date < datetime.datetime(2015, 7, 1)]
NameError: name 'df' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\

In [ ]:
X_train.shape, X_val.shape

ERROR! Session/line number was not unique in database. History logging moved to new session 328


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-eec90bb64b1e>", line 1, in <module>
    X_train.shape, X_val.shape
NameError: name 'X_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_recor

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-eec90bb64b1e>", line 1, in <module>
    X_train.shape, X_val.shape
NameError: name 'X_train' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3337, in 

In [ ]:
# Normalize output and determine wether to use log_output 
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 329
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-f8fb65afefb1>", line 6, in <module>
    max_log_y = np.max(np.log(df[y_out_columns])).values
NameError: name 'np' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\Ap

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-f8fb65afefb1>", line 6, in <module>
    max_log_y = np.max(np.log(df[y_out_columns])).values
NameError: name 'np' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshel

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 316, in wrapped
    return f(*args, **kwargs)
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", l

## Data from hyper opt - Iterations

In [ ]:
iterations_from_hyperopt = pd.read_csv('hyper_opt_iterations.csv')
iterations_from_hyperopt.sort_values('loss')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 330
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-aeff0f2b637e>", line 1, in <module>
    iterations_from_hyperopt = pd.read_csv('hyper_opt_iterations.csv')
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Use

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-aeff0f2b637e>", line 1, in <module>
    iterations_from_hyperopt = pd.read_csv('hyper_opt_iterations.csv')
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\i

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-aeff0f2b637e>", line 1, in <module>
    iterations_from_hyperopt = pd.read_csv('hyper_opt_iterations.csv')
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\i

In [ ]:
# We print the best parameters found
iterations_from_hyperopt.loc[85,:]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 331
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-b8199a592e55>", line 2, in <module>
    iterations_from_hyperopt.loc[85,:]
NameError: name 'iterations_from_hyperopt' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\6006877

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-b8199a592e55>", line 2, in <module>
    iterations_from_hyperopt.loc[85,:]
NameError: name 'iterations_from_hyperopt' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactive

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-b8199a592e55>", line 2, in <module>
    iterations_from_hyperopt.loc[85,:]
NameError: name 'iterations_from_hyperopt' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactive

## Model

In [ ]:
from lightgbm import LGBMRegressor

ERROR! Session/line number was not unique in database. History logging moved to new session 332


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-237e8b2a5f63>", line 1, in <module>
    from lightgbm import LGBMRegressor
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\lightgbm\__init__.py", line 8, in <module>
    from .basic import Booster, Dataset, register_logger
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\lightgbm\basic.py", line 14, in <module>
    import numpy as np
  File "C:\Users\60068779\AppData\Roaming\Python\Python37\site-packages\numpy\__init__.py", line 156, in <module>
    from . import random
  File "C:\Users\60068779\AppData\Roaming\Python\Python37\site-packages\numpy\random\__init__.py", line 179, in <module>
    from . import _pickle
  File "C:\User

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-237e8b2a5f63>", line 1, in <module>
    from lightgbm import LGBMRegressor
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\lightgbm\__init__.py", line 8, in <module>
    from .basic import Booster, Dataset, register_logger
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\lightgbm\basic.py", line 14, in <module>
    import numpy as np
  File "C:\Users\60068779\AppData\Roaming\Python\Python37\site-packages\numpy\__init__.py", line 156, in <module>
    from . import random
  File "C:\Users\60068779\AppData\Roaming\Python\Python37\site-packages\numpy\random\__init__.py", line 179, in <module>
    from . import _pickle
  File "C:\User

In [ ]:
# Los valores presentados a continuación son los finales que entregan el mejor score privado en la competencia.
# Son una modificación a mano de los parametros que entregó el optimizador. 
# Las modificaciones fueron para regularizar: subir el reg_lambda, bajar los n_estimators y subir el LR, etc.
min_child_samples=5
n_estimators=1000
learning_rate=0.0516
max_depth = 500
num_leaves=53
min_child_samples= 200 #194
reg_lambda=10 #20 #10 #0.000000
reg_alpha=1.000000
colsample_bytree=0.519264
min_child_weight=0.000000
n_estimators=1000

model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, num_leaves=num_leaves, reg_lambda=reg_lambda,
                     reg_alpha=reg_alpha, colsample_bytree=colsample_bytree, min_child_weight=min_child_weight,n_jobs=8)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 333
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-acfb567418ea>", line 16, in <module>
    model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, num_leaves=num_leaves, reg_lambda=reg_lambda,
NameError: name 'LGBMRegressor' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_trac

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-acfb567418ea>", line 16, in <module>
    model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, num_leaves=num_leaves, reg_lambda=reg_lambda,
NameError: name 'LGBMRegressor' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most re

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-acfb567418ea>", line 16, in <module>
    model = LGBMRegressor(min_child_samples=min_child_samples, n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, num_leaves=num_leaves, reg_lambda=reg_lambda,
NameError: name 'LGBMRegressor' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most re

In [ ]:
# Fit params
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'l2', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': cat_vars
           }

ERROR! Session/line number was not unique in database. History logging moved to new session 334


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-7d68347dc8d1>", line 4, in <module>
    "eval_set" : [(X_val, y_val.reshape(-1))],
NameError: name 'X_val' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 1169

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-7d68347dc8d1>", line 4, in <module>
    "eval_set" : [(X_val, y_val.reshape(-1))],
NameError: name 'X_val' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", 

In [ ]:
# Fit model
model.fit(X_train, y_train.reshape(-1), **fit_params)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 335
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-0b4c898a057d>", line 2, in <module>
    model.fit(X_train, y_train.reshape(-1), **fit_params)
NameError: name 'model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\6006877

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-0b4c898a057d>", line 2, in <module>
    model.fit(X_train, y_train.reshape(-1), **fit_params)
NameError: name 'model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactive

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-0b4c898a057d>", line 2, in <module>
    model.fit(X_train, y_train.reshape(-1), **fit_params)
NameError: name 'model' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactive

# Analisis de los resultados

## Metrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [ ]:
# No sacamos lo de train para acelerar la notebook.
if log_output:
    #y_pred_train = np.exp(model.predict(X_train, verbose=1)*max_log_y)
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test, verbose=1)*max_log_y)
else:
    #y_pred_train = model.predict(X_train, verbose=1)*y_std + y_mean
    y_pred = model.predict(X_val, verbose=1)*y_std + y_mean
    y_pred_test = model.predict(X_test, verbose=1)*y_std + y_mean

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 336
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-c2935352d6aa>", line 4, in <module>
    y_pred = np.exp(model.predict(X_val, verbose=1)*max_log_y)
NameError: name 'np' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 1193, in structured_traceback
    tb_offset)
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 1150, in format_exception_as_a_whole
    last_unique, recursion_repeat = find_recursion(orig_etype, evalue, records)
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 451, in find_recursion
    return len(records), 0
TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_re

In [ ]:
# Validación
val_score = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))
print(val_score)

ERROR! Session/line number was not unique in database. History logging moved to new session 337


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-4e052c8e8371>", line 2, in <module>
    val_score = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))
NameError: name 'np' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearn

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-4e052c8e8371>", line 2, in <module>
    val_score = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))
NameError: name 'np' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearn

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-4e052c8e8371>", line 2, in <module>
    val_score = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))
NameError: name 'np' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearn

# Submit a la competencia

In [ ]:
sample_csv = pd.read_csv('sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

sample_csv.to_csv(f'submision_lightgbm_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}-SCORE-{val_score}.csv', index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 338
Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-a0d199cbe28a>", line 1, in <module>
    sample_csv = pd.read_csv('sample_submission.csv')
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppDa

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-a0d199cbe28a>", line 1, in <module>
    sample_csv = pd.read_csv('sample_submission.csv')
NameError: name 'pd' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.p

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



TypeError: object of type 'NoneType' has no len()

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLearning\lib\site-packages\IPython\core\ultratb.py", line 316, in wrapped
    return f(*args, **kwargs)
  File "C:\Users\60068779\AppData\Local\Continuum\anaconda3\envs\DeepLear